In [ ]:
# run this code to login to https://okpy.org/ and setup the assignment for submission
from ist256 import okclient
ok = okclient.Lab()

# In-Class Coding Lab: Data Analysis with Pandas

In this lab, we will perform a data analysis on the **RMS Titanic** passenger list. The RMS Titanic is one of the most famous ocean liners in history. On April 15, 1912 it sank after colliding with an iceberg in the North Atlantic Ocean. To learn more, read here: https://en.wikipedia.org/wiki/RMS_Titanic 

Our goal today is to perform a data analysis on a subset of the passenger list. We're looking for insights as to which types of passengers did and didn't survive. Women? Children? 1st Class Passengers? 3rd class? Etc. 

I'm sure you've heard the expression often said during emergencies: "Women and Children first" Let's explore this data set and find out if that's true!

Before we begin you should read up on what each of the columns mean in the data dictionary. You can find this information on this page: https://www.kaggle.com/c/titanic/data 


## Loading the data set

First we load the dataset into a Pandas `DataFrame` variable. The `sample(10)` method takes a random sample of 10 passengers from the data set.

In [1]:
import pandas as pd
import numpy as np

# this turns off warning messages
import warnings
warnings.filterwarnings('ignore')

passengers = pd.read_csv('CCL-titanic.csv')
passengers.sample(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
767,768,0,3,"Mangan, Miss. Mary",female,30.5,0,0,364850,7.7500,NaN,Q
447,448,1,1,"Seward, Mr. Frederic Kimber",male,34.0,0,0,113794,26.5500,NaN,S
552,553,0,3,"O'Brien, Mr. Timothy",male,NaN,0,0,330979,7.8292,NaN,Q
391,392,1,3,"Jansson, Mr. Carl Olof",male,21.0,0,0,350034,7.7958,NaN,S
360,361,0,3,"Skoog, Mr. Wilhelm",male,40.0,1,4,347088,27.9000,NaN,S
289,290,1,3,"Connolly, Miss. Kate",female,22.0,0,0,370373,7.7500,NaN,Q
246,247,0,3,"Lindahl, Miss. Agda Thorilda Viktoria",female,25.0,0,0,347071,7.7750,NaN,S
573,574,1,3,"Kelly, Miss. Mary",female,NaN,0,0,14312,7.7500,NaN,Q
779,780,1,1,"Robert, Mrs. Edward Scott (Elisabeth Walton Mc...",female,43.0,0,1,24160,211.3375,B3,S
830,831,1,3,"Yasbeck, Mrs. Antoni (Selini Alexander)",female,15.0,1,0,2659,14.4542,NaN,C


## How many survived?

One of the first things we should do is figure out how many of the passengers in this data set survived. Let's start with isolating just the `'Survivied'` column into a series:

In [2]:
passengers['Survived'].sample(10)

344    0
71     0
444    1
580    1
719    0
424    0
853    1
360    0
162    0
564    0
Name: Survived, dtype: int64

There's too many to display so we just display a random sample of 10 passengers. 

- 1 means the passenger survivied
- 0 means the passenger died

What we really want is to count the number of survivors and deaths. We do this by querying the `value_counts()` of the `['Survived']` column, which returns a `Series` of counts, like this:

In [3]:
passengers['Survived'].value_counts()

0    549
1    342
Name: Survived, dtype: int64

Only 342 passengers survived, and 549 perished. Let's observe this same data as percentages of the whole. We do this by adding the `normalize=True` named argument to the `value_counts()` method.

In [4]:
passengers['Survived'].value_counts(normalize=True)

0    0.616162
1    0.383838
Name: Survived, dtype: float64

**Just 38% of passengers in this dataset survived.**

### Now you Try it!

**FIRST** Write a Pandas expression to display counts of males and female passengers using the `Sex` variable:

In [5]:
# todo write code here
passengers['Sex'].value_counts()


male      577
female    314
Name: Sex, dtype: int64

**NEXT** Write a Pandas expression to display male /female passenger counts as a percentage of the whole number of passengers in the data set.

In [6]:
# todo write code here
passengers['Sex'].value_counts(normalize=True)

male      0.647587
female    0.352413
Name: Sex, dtype: float64

If you got things working, you now know that **35% of passengers were female**.

## Who survivies? Men or Women?

We now know that 35% of the passengers were female, and 65% we male. 

**The next thing to think about is how do survivial rates affect these numbers? **

If the ratio is about the same for surviviors only, then we can conclude that your **Sex** did not play a role in your survival on the RMS Titanic. 

Let's find out.

In [7]:
survivors = passengers[passengers['Survived'] ==1]
survivors['PassengerId'].count()

342

Still **342** like we discovered originally. Now let's check the **Sex** split among survivors only:

In [8]:
survivors['Sex'].value_counts()

female    233
male      109
Name: Sex, dtype: int64

WOW! That is a huge difference! But you probably can't see it easily. Let's represent it in a `DataFrame`, so that it's easier to visualize:

In [9]:
sex_all_series = passengers['Sex'].value_counts()
sex_survivor_series = survivors['Sex'].value_counts()

sex_comparision_df = pd.DataFrame({ 'AllPassengers' : sex_all_series, 'Survivors' : sex_survivor_series })
sex_comparision_df['SexSurvivialRate'] = sex_comparision_df['Survivors'] / sex_comparision_df['AllPassengers']
sex_comparision_df

,AllPassengers,Survivors,SexSurvivialRate
female,314,233,0.742038
male,577,109,0.188908


 **So, females had a 74% survival rate. Much better than the overall rate of 38%**
 
We should probably briefly explain the code above. 

- The first two lines get a series count of all passengers by Sex (male / female) and count of survivors by sex
- The third line creates a Pandas DataFrame. Recall a pandas dataframe is just a dictionary of series. We have two keys 'AllPassengers' and 'Survivors'
- The  fourth line creates a new column in the dataframe which is just the survivors / all passengers to get the rate of survival for that Sex.

## Feature Engineering: Adults and Children

Sometimes the variable we want to analyze is not readily available, but can be created from existing data. This is commonly referred to as **feature engineering**. The name comes from machine learning where we use data called *features* to predict an outcome. 

Let's create a new feature called `'AgeCat'` as follows:

- When **Age** <=18 then 'Child'
- When **Age** >18 then 'Adult'

This is easy to do in pandas. First we create the column and set all values to `np.nan` which means 'Not a number'. This is Pandas way of saying no value. Then we set the values based on the rules we set for the feature.

In [10]:
passengers['AgeCat'] = np.nan # Not a number
passengers['AgeCat'][ passengers['Age'] <=18 ] = 'Child'
passengers['AgeCat'][ passengers['Age'] > 18 ] = 'Adult'
passengers.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeCat
716,717,1,1,"Endres, Miss. Caroline Louise",female,38.0,0,0,PC 17757,227.5250,C45,C,Adult
758,759,0,3,"Theobald, Mr. Thomas Leonard",male,34.0,0,0,363294,8.0500,NaN,S,Adult
713,714,0,3,"Larsson, Mr. August Viktor",male,29.0,0,0,7545,9.4833,NaN,S,Adult
334,335,1,1,"Frauenthal, Mrs. Henry William (Clara Heinshei...",female,NaN,1,0,PC 17611,133.6500,NaN,S,NaN
450,451,0,2,"West, Mr. Edwy Arthur",male,36.0,1,2,C.A. 34651,27.7500,NaN,S,Adult


Let's get the count and distrubutions of Adults and Children on the passenger list.

In [11]:
passengers['AgeCat'].value_counts()

Adult    575
Child    139
Name: AgeCat, dtype: int64

And here's the percentage as a whole:

In [12]:
passengers['AgeCat'].value_counts(normalize=True)

Adult    0.805322
Child    0.194678
Name: AgeCat, dtype: float64

So close to **80%** of the passengers were adults. Once again let's look at the ratio of `AgeCat` for survivors only. If your age has no bearing of survivial, then the rates should be the same. 

Here are the counts of Adult / Children among the survivors only:

In [13]:
survivors = passengers[passengers['Survived'] ==1]
survivors['AgeCat'].value_counts()

Adult    220
Child     70
Name: AgeCat, dtype: int64

### Now You Try it!

Calculate the `AgeCat` survival rate, similar to how we did for the `SexSurvivalRate`. 

In [14]:
agecat_all_series = passengers['AgeCat'].value_counts()
agecat_survivor_series = survivors['AgeCat'].value_counts()

# todo make a data frame, add AgeCatSurvivialRate column, display dataframe 
AgeCat_df = pd.DataFrame({ 'AllPassengers' : agecat_all_series, 'AgeCat' : agecat_survivor_series })
AgeCat_df['AgeCat'] = AgeCat_df['AgeCat'] / AgeCat_df['AllPassengers']
AgeCat_df

,AgeCat,AllPassengers
Adult,0.382609,575
Child,0.503597,139


**So, children had a 50% survival rate, better than the overall rate of 38%**

## So, women and children first?

It looks like the RMS really did have the motto: "Women and Children First."

Here are our insights. We know:

- If you were a passenger, you had a 38% chance of survival.
- If you were a female passenger, you had a 74% chance of survival.
- If you were a child passenger, you had a 50% chance of survival. 


### Now you try it for Passenger Class

Repeat this process for `Pclass` The passenger class variable. Display the survival rates for each passenger class. What does the information tell you about passenger class and survival rates?

I'll give you a hint... "Money Talks"


In [15]:
# todo: repeat the analysis in the previous cell for Pclass 
pclass_all_series = passengers['Pclass'].value_counts()
pclass_survivor_series = survivors['Pclass'].value_counts()
# todo make a data frame, add AgeCatSurvivialRate column, display dataframe 
Pclass_df = pd.DataFrame({ 'AllPassengers' : pclass_all_series, 'Pclass' : pclass_survivor_series })
Pclass_df['Pclass'] = Pclass_df['Pclass'] / Pclass_df['AllPassengers']
Pclass_df

,AllPassengers,Pclass
1,216,0.629630
2,184,0.472826
3,491,0.242363


## Metacognition

Please answer the following questions. This should be a personal narrative, in your own voice. Answer the questions by double clicking on the question and placing your answer next to the Answer: prompt.


1. Record any questions you have about this lab that you would like to ask in recitation. It is expected you will have questions if you did not complete the code sections correctly.  Learning how to articulate what you do not understand is an important skill of critical thinking. 

Answer: My question was about the final Now You try. The labels seem to be inverted, is there a way to fix this or is it working like it is supposed to?

2. What was the most difficult aspect of completing this lab? Least difficult?  

Answer: The most difficult aspect of this lab was the final now you try. I am having trouble trying to figure out how to switch around the labels.

3. What aspects of this lab do you find most valuable? Least valuable?

Answer: I find this lab to be very helpful because it is a very organized way to cite rates through the list of passengers. I feel that everything in this CCL is very valuable.

4. Rate your comfort level with this week's material so far.   

1 ==> I can do this on my own and explain how to do it.   
2 ==> I can do this on my own without any help.   
3 ==> I can do this with help or guidance from others. If you choose this level please list those who helped you.   
4 ==> I don't understand this at all yet and need extra help. If you choose this please try to articulate that which you do not understand.   

Answer: I am at a 2 with this currently. I can do all of it without any help but I would have to do it using trial and error. It is not something I would be able to explain to someone on paper.



In [ ]:
# to save and turn in your work, execute this cell. Your latest submission will be graded. 
ok.submit()